In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import sys
sys.path.append('./modules')
from datasets import MalariaDataset

In [ ]:
transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((128,128)),
            transforms.ToTensor(),
            # transforms.Lambda(lambda x: x.repeat(3,1,1)),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            # transforms.RandomHorizontalFlip(p=1),
            # transforms.RandomRotation((90,90),expand=False, center=None, fill=0),

    ])

training_data = MalariaDataset(annotations_file = r'.\malaria\train_malaria.csv', img_dir = r'.\malaria\train', transform=transform)
test_data = MalariaDataset(annotations_file = r'.\malaria\test_malaria.csv', img_dir = r'.\malaria\test', transform=transform)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
classes = ('Normal', 'Parasitada')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=64*32*32, out_features=128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(in_features=128, out_features=2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 64*32*32)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

net = Net()

# Print das entradas e saídas de cada camada
summary(net, (3, 128, 128))

In [ ]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
from torch.utils.data import DataLoader

batch_sizes = [16,32,64]

In [ ]:
for batch_size in batch_sizes:
    
    trainloader = torch.utils.data.DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=2)
    testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)
    
    num_epochs = 6

    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if i % 1000 == 999:
                with torch.no_grad():
                    total = 0
                    correct = 0
                    for data in testloader:
                        images, labels = data
                        outputs = net(images)
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                        
                print(f'Epoch {epoch + 1}, batch {i + 1}: loss {running_loss / 2000:.3f}, accuracy {correct / total:.3f}')
                running_loss = 0.0
                
    print('Finished Training')
    
    PATH = f'./models/malaria_3x3_{batch_size}.pth'
    torch.save(net.state_dict(), PATH)
    
    net = Net()
    net.load_state_dict(torch.load(PATH))
    
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Batch_size: {batch_size} - zAcurácia da rede neural em 8267 imagens: {100 * correct / total} %')